In [625]:
from PIL import Image
import numpy as np
import os
import timeit
from shutil import copyfile
import time
from collections import Counter

dims = (600,400)
work_path = os.getcwd()
in_p = work_path+'/ADE20K_2016_07_26/images/training/a/airlock'
out_p = work_path+'/demo/'
clean_path = lambda f_p: '/'.join(f_p.split('/')[5:])

# get rgbs
c_v = lambda img: img.getcolors()
get_colors = lambda img: [i[1] for i in c_v(img)]
get_percentile = lambda img: ([i[0]/sum([i[0] for i in c_v(img)]) for i in c_v(p)])

# quick pixel guess
pixel_same = lambda pix,colors: True if (pix in colors) else False
pixel_diff = lambda pix,colors: sum([(pix[i]-colors[i])**2 for i in range(0,2)])
win_choice = lambda old, new: old if old[0]<=new[0] else new

def guess_pix(pix, color_range):
    if pixel_same(pix,color_range):
        return pix
    else:
        win_val = (pixel_diff(pix, color_range[0]),0)
        for i in range(1, len(color_range)):
                win_val = win_choice(win_val, (pixel_diff(pix, color_range[i]),i))
                return color_range[win_val[1]]

# difficult pixel guess
check_pix = lambda fl,l: l[4] if len(fl) == 0 else fl
get_nearby_pix = lambda arr,r,c: [tuple(arr[r,c]) for r,c in ((i,j) for i in range(-1,2) for j in range(-1,2))]
keep_real_pix = lambda l, cl: check_pix([rgb for rgb in l if rgb in cl],l)

def assign_pixels(arr,colors,s):
    new_arr = arr
    for i in range(s[0]): 
        for j in range(s[1]):
            if j not in [0,s[0]] or i not in [0,s[1]]:
                new_arr[i][j] = Counter(keep_real_pix(get_nearby_pix(arr,1,1),colors)).most_common(1)[0][0]
    return new_arr

# save jpg file            
def save_jpg(paths,dim):
    os.makedirs(os.path.dirname(paths[1]+paths[0]), exist_ok=True)
    with Image.open(paths[0]) as im:
        im.resize(dim).save(paths[1]+paths[0])
        
# save segment file
def save_segment(paths,dim):
    with Image.open(paths[0]) as im:
        s = np.array(im).shape
        if (s[0]<dim[0]) or (s[1]<dim[1]):
            im.resize(dim).save(paths[1]+paths[0])
        else:
            colors = get_colors(im)
            im.thumbnail(dim, Image.ANTIALIAS)
            im = np.array(im)
            s = im.shape
            if True:
                im = assign_pixels(im,colors,s)
            else:
                im = np.array([guess_pix(tuple(p),colors) for p in im.reshape(s[0]*s[1],3)])
            im = Image.fromarray(im.reshape(s).astype(np.uint8)).save(paths[1]+paths[0])
            
# file navigation
def get_file_paths(w_path):
    return (clean_path(p) for p in [os.path.join(dirpath, f)
        for dirpath, dirnames, files in os.walk(w_path)
        for f in files if f.endswith('.jpg')])

def parse_folders(w_path):
    start = time.time()
    all_files = get_file_paths(w_path)
    for f in all_files:
        #resize main photo
        print('/'.join(f.split('/')[2:]),end='')
        save_jpg([f, out_p],dims)
        
        #resize segment photo
        f_seg=f[:-4]+'_seg.png'
        save_segment([f_seg,out_p], dims)   
        
        #copy text file
        f_txt=f[:-4]+'_atr.txt'
        copyfile(f_txt,out_p+f_txt)
        print(' [\x1b[34mDONE\x1b[0m]')
    end = time.time()
    print(f'\u001b[1mTIME ELAPSED:\x1b[0m \x1b[34m{end-start}\x1b[0m')

In [ ]:
parse_folders(in_p)

training/a/airlock/ADE_train_00001044.jpg [DONE]
training/a/airlock/ADE_train_00001045.jpg [DONE]
training/a/airlock/ADE_train_00001047.jpg [DONE]
training/a/airlock/ADE_train_00001046.jpg [DONE]
training/a/airlock/ADE_train_00001042.jpg [DONE]
training/a/airlock/ADE_train_00001043.jpg

In [ ]:
for i in range(s[0]): 
    for j in range(s[1]):
        if j not in [0,s[0]] or i not in [0,s[1]]:
            colors = practice_image[]
            practice_image[i][j]